In [3]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install transformers

     ---------------------------------------- 0.0/44.1 kB ? eta -:--:--
     ---------------------------------------- 44.1/44.1 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.0 MB 7.9 MB/s eta 0:00:02
   -- ------------------------------------- 0.6/10.0 MB 7.4 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/10.0 MB 18.9 MB/s eta 0:00:01
   ------------------- -------------------- 4.8/10.0 MB 27.8 MB/s eta 0:00:01
   ---------------------------------- ----- 8.6/10.0 MB 42.3 MB/s eta 0:00:01
   ---------------------------------------  10.0/10.0 MB 45.7 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 40.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/286.3 kB ? eta -:--:--
   --------------------------------------- 286.3/286.3 kB 17.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------


[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import torch
from torch import nn
from transformers import BertConfig
from torchdiffeq import odeint


class AttentionODEFunc(nn.Module):
    def __init__(self, head_dim):
        super(AttentionODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(head_dim, head_dim),
            nn.ReLU(),
            nn.Linear(head_dim, head_dim)
        )

    def forward(self, t, attention_scores):
        return self.net(attention_scores)


class ODEAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads=8, time_steps=10):
        super(ODEAttention, self).__init__()
        self.num_heads = num_heads
        self.hidden_dim = hidden_dim
        self.head_dim = hidden_dim // num_heads
        self.time_steps = time_steps
        self.scaling_factor = self.head_dim ** 0.5

        self.ode_func = AttentionODEFunc(self.head_dim)

    def forward(self, Q, K, V):
        batch_size, seq_length, _ = Q.size()
        assert Q.size(-1) == self.hidden_dim, f"Expected hidden size {self.hidden_dim}, got {Q.size(-1)}"

        Q = Q.view(batch_size, seq_length, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, seq_length, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, seq_length, self.num_heads, self.head_dim).permute(0, 2, 1, 3)

        Q = Q.reshape(-1, seq_length, self.head_dim)
        K = K.reshape(-1, seq_length, self.head_dim)
        V = V.reshape(-1, seq_length, self.head_dim)

        initial_scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scaling_factor

        initial_scores = initial_scores.unsqueeze(-1)

        initial_scores = initial_scores.view(-1, self.head_dim)

        t_span = torch.linspace(0, 1, steps=self.time_steps).to(Q.device)

        evolved_scores = odeint(self.ode_func, initial_scores, t_span)

        evolved_scores = evolved_scores[-1].view(-1, seq_length, seq_length)

        batch_heads = evolved_scores.size(0)
        expected_size = batch_heads * seq_length * seq_length
        assert evolved_scores.numel() == expected_size, (
            f"Shape mismatch: expected {expected_size}, got {evolved_scores.numel()}"
        )

        attention_weights = torch.softmax(evolved_scores, dim=-1)

        output = torch.matmul(attention_weights, V)

        output = output.view(batch_size, self.num_heads, seq_length, self.head_dim).permute(0, 2, 1, 3)
        output = output.reshape(batch_size, seq_length, self.hidden_dim)
        return output, attention_weights


class ODEBertSelfAttention(nn.Module):
    def __init__(self, config):
        super(ODEBertSelfAttention, self).__init__()
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.ode_attention = ODEAttention(config.hidden_size, config.num_attention_heads)
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

    def forward(self, hidden_states):
        Q = self.query(hidden_states)
        K = self.key(hidden_states)
        V = self.value(hidden_states)
        context_layer, attention_weights = self.ode_attention(Q, K, V)
        context_layer = self.dense(context_layer)
        context_layer = self.dropout(context_layer)
        return context_layer, attention_weights


class ODEBertModel(nn.Module):
    def __init__(self, config):
        super(ODEBertModel, self).__init__()
        self.embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
        self.encoder = nn.ModuleList(
            [ODEBertSelfAttention(config) for _ in range(config.num_hidden_layers)]
        )

    def forward(self, input_ids, attention_mask):
        embedded_inputs = self.embeddings(input_ids)
        hidden_states = embedded_inputs
        for layer in self.encoder:
            hidden_states, _ = layer(hidden_states)
        return {"last_hidden_state": hidden_states}


def test_self_attention():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    config = BertConfig(hidden_size=768, num_attention_heads=12, attention_probs_dropout_prob=0.1)
    ode_self_attention = ODEBertSelfAttention(config).to(device)

    # Synthetic input
    batch_size = 4
    seq_length = 128
    hidden_states = torch.rand(batch_size, seq_length, config.hidden_size).to(device)

    # Forward pass
    context_layer, attention_weights = ode_self_attention(hidden_states)
    print(f"Context Layer Shape: {context_layer.shape}")
    print(f"Attention Weights Shape: {attention_weights.shape}")


def test_bert_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    config = BertConfig(
        vocab_size=30522,
        hidden_size=768,
        num_attention_heads=12,
        num_hidden_layers=4,
        attention_probs_dropout_prob=0.1,
    )
    ode_bert_model = ODEBertModel(config).to(device)

    batch_size = 4
    seq_length = 128
    input_ids = torch.randint(0, config.vocab_size, (batch_size, seq_length)).to(device)
    attention_mask = torch.ones((batch_size, seq_length)).to(device)

    outputs = ode_bert_model(input_ids=input_ids, attention_mask=attention_mask)
    print(f"Last Hidden State Shape: {outputs['last_hidden_state'].shape}")


# testinggg
if __name__ == "__main__":
    print("Testing ODEBertSelfAttention...")
    test_self_attention()
    print("\nTesting ODEBertModel...")
    test_bert_model()


Testing ODEBertSelfAttention...
Context Layer Shape: torch.Size([4, 128, 768])
Attention Weights Shape: torch.Size([48, 128, 128])

Testing ODEBertModel...
Last Hidden State Shape: torch.Size([4, 128, 768])


In [6]:
class ODEBertForTextClassification(ODEBertModel):
    def __init__(self, config, num_labels):
        super().__init__(config)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs["last_hidden_state"][:, 0, :]
        cls_output = self.dropout(cls_output)
        logits = self.classifier(cls_output)
        loss = None
        if labels is not None:
            criterion = nn.CrossEntropyLoss()
            loss = criterion(logits, labels)
        return {"loss": loss, "logits": logits}

In [7]:
def test_text_classification():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    config = BertConfig(
        vocab_size=30522,
        hidden_size=768,
        num_attention_heads=12,
        num_hidden_layers=4,
        attention_probs_dropout_prob=0.1,
        hidden_dropout_prob=0.1,
    )
    num_labels = 2  

    model = ODEBertForTextClassification(config, num_labels).to(device)

    batch_size = 4
    seq_length = 128
    input_ids = torch.randint(0, config.vocab_size, (batch_size, seq_length)).to(device)
    attention_mask = torch.ones((batch_size, seq_length)).to(device)
    labels = torch.randint(0, num_labels, (batch_size,)).to(device)  

    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

    print(f"Loss: {outputs['loss']}")
    print(f"Logits Shape: {outputs['logits'].shape}") 


#  testing
if __name__ == "__main__":
    print("Testing ODEBertForTextClassification on GPU...")
    test_text_classification()

Testing ODEBertForTextClassification on GPU...
Loss: 0.6983002424240112
Logits Shape: torch.Size([4, 2])


In [8]:
from transformers import BertTokenizer, BertConfig


In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
config = BertConfig.from_pretrained("bert-base-uncased")

In [12]:
num_labels = 2  
model = ODEBertForTextClassification(config, num_labels)

In [23]:
from datasets import load_dataset
dataset = load_dataset("imdb")


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

C:\Users\ADMIN\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\datasets--imdb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [27]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [29]:
from torch.utils.data import DataLoader


In [31]:
train_data = tokenized_datasets["train"].shuffle(seed=42).select(range(200))
test_data = tokenized_datasets["test"].shuffle(seed=42).select(range(50))
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=8)


In [33]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
epochs = 3

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["labels"])
        loss = outputs["loss"]
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}")



KeyboardInterrupt



In [37]:
from tqdm import tqdm

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
epochs = 100

for epoch in range(epochs):
    model.train()
    total_loss = 0

    # Wrap the dataloader with tqdm for progress tracking
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}", unit="batch")

    for batch in progress_bar:
        optimizer.zero_grad()
        outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["labels"])
        loss = outputs["loss"]
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Update the progress bar with the current loss
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}")


Epoch 1/100: 100%|██████████████████████████████████████████████████████| 25/25 [20:58<00:00, 50.35s/batch, loss=0.697]


Epoch 1, Average Loss: 0.6947


Epoch 2/100: 100%|██████████████████████████████████████████████████████| 25/25 [20:35<00:00, 49.43s/batch, loss=0.676]


Epoch 2, Average Loss: 0.6922


Epoch 3/100: 100%|██████████████████████████████████████████████████████| 25/25 [20:21<00:00, 48.85s/batch, loss=0.685]


...

Epoch 15, Average Loss: 0.1321


Epoch 16/100: 100%|████████████████████████████████████████████████████| 25/25 [19:23<00:00, 46.56s/batch, loss=0.0096]


Epoch 16, Average Loss: 0.0543


Epoch 17/100:   0%|                                                                          | 0/25 [00:29<?, ?batch/s]


KeyboardInterrupt: 